In [5]:
import numpy as np
from queue import Queue
arr = np.genfromtxt('Day11.txt',delimiter = ',',dtype=np.int64)

In [6]:
def indexing(listt,index):
    inxlist = []
    global arr1
    for i, ss in enumerate(listt):
        if ss == '0': #position mode
            inxlist.append(arr1[index+i+1])
        elif ss =='1': #immediate mode
            inxlist.append(index+i+1)
        else: #ss == '2'
            global base
            inxlist.append(arr1[index+i+1]+base)
    return inxlist

def arrOdict(inxlist):
    global arr1
    global lgmem
    vallist = []
    for inx in inxlist:
        if inx > (len(arr1)-1):
            if inx not in lgmem.keys():
                lgmem[inx] = 0
                vallist.append(lgmem[inx])
            else:
                vallist.append(lgmem[inx])
        else:
            vallist.append(arr1[inx])
    return vallist

def setequal(inxequal,num):
    global arr1
    global lgmem
    if inxequal > (len(arr1)-1):
        lgmem[inxequal] = num
    else:
        arr1[inxequal] = num
    return None

def opcodeAct(index,inxlist,opcode):
    global arr1
    global inpt
    global q
    vallist = arrOdict(inxlist)
    if opcode == 1:
        setequal(inxlist[2],vallist[0] + vallist[1])
        index += 4
    elif opcode == 2:
        setequal(inxlist[2],vallist[0] * vallist[1])
        index += 4
    elif opcode == 4:
        q.put(vallist[0])
        index += 2
    elif opcode == 3:
        setequal(inxlist[0],inpt)
        index += 2
    elif opcode == 5:
        if vallist[0]: #If non-zero is true
            index = vallist[1]
        else:
            index += 3
    elif opcode == 6:
        if not vallist[0]:
            index = vallist[1]
        else:
            index += 3
    elif opcode == 7:
        if vallist[0] < vallist[1]:
            setequal(inxlist[2],1)
        else:
            setequal(inxlist[2],0)
        index += 4
    elif opcode == 8:
        if vallist[0] == vallist[1]:
            setequal(inxlist[2],1)
        else:
            setequal(inxlist[2],0)
        index += 4
    elif opcode == 9:
        global base
        base += vallist[0]
        index += 2
    return index

def day11(arr):
    global arr1
    global inpt
    global q
    arr1 = arr.copy()
    q = Queue()
    mapp = {}
    currpt = [0,0]
    currdir = 1 #up/right is 1, down/left is -1
    mapp[tuple(currpt)] = 0
    count = 0
    index = 0
    while arr1[index] != 99:
        if arr1[index] in [1,2,7,8]:
            index = opcodeAct(index,arr1[index+1:index+4],arr1[index])
        elif arr1[index] in [5,6]:
            index = opcodeAct(index,arr1[index+1:index+3],arr1[index])
        elif arr1[index] in [3,4,9]:
            index = opcodeAct(index,arr1[index+1:index+2],arr1[index])
        else: #5 digit opcode
            s = str(arr1[index])
            s = s.rjust(5,'0')
            s = s[::-1]
            if int(s[0]) in [1,2,7,8]:
                inxlist = indexing(s[2:],index)
            elif int(s[0]) in [5,6]:
                inxlist = indexing(s[2:4],index)
            elif int(s[0]) in [3,4,9]:
                inxlist = indexing(s[2:3],index)
            index = opcodeAct(index,inxlist,int(s[0]))
        if len(q.queue) == 2:
            color = q.get()
            direct = q.get() #left is 0, right is 1 
            if direct == 0: #left equal to -1
                direct = -1
            mapp[tuple(currpt)] = color #paint current panel first
            painted = 1
            if count%2: #if remainder is 1, left/right
                if currdir == 1: #right
                    currpt[1] -= direct #go up or down
                    currdir = -direct #1 if up, -1 if down, left is up, right is down
                else: #left
                    currpt[1] += direct
                    currdir = direct
            else: #if remainder is 0, currently up/down
                if currdir == 1: #up
                    currpt[0] += direct #go left or right
                    currdir = direct #1 is right, -1 is left
                else: #down
                    currpt[0] -= direct
                    currdir = -direct
            if tuple(currpt) not in mapp:
                mapp[tuple(currpt)] = 0
                painted = 0
            inpt = mapp[tuple(currpt)]
            count += 1
    return mapp,painted

In [7]:
base = 0
lgmem = {}
inpt = 0
mapp,painted = day11(arr)

In [8]:
if painted == 1:
    print(len(mapp))
else:
    print(len(mapp)-1)

1863


In [97]:
#Part 2
base = 0
lgmem = {}
inpt = 1
mapp,painted = day11(arr)

250

In [98]:
coord = np.array([list(item) for item in mapp.keys()])
mx = np.max(abs(coord[:]))
coord += mx
paper = np.zeros(shape=(2*(mx+1),2*(mx+1)))
painting = np.array([*mapp.values()])
for xy,paint in zip(coord,painting):
    paper[xy[0],xy[1]] = paint

In [100]:
from PIL import Image
img = Image.fromarray(paper.astype('uint8')*255)
img.show()